In [2]:
%matplotlib inline
from ipywidgets import widgets
from ipywidgets import Layout
from IPython import display
import configparser
import pickle
import numpy as np
import serial
import time
import sys
import glob

gstring="global "

N=8 #number of pulsers
N_out=8 #number of ouputs

N_save=6 #number of save buttons

## Configuration File reader 
ini_file='pulser.INI'
config = configparser.ConfigParser()
config.read(ini_file)

data_input_style = "<style>.data_input input { background-color:#D0F0D0 !important; }"+\
                   ".data_wrong input { background-color:#F00000 !important; }"+\
                   ".data_display input { foreground-color:#000000 !important; }</style>"

clk_s=100000000 #clock cycles per second
connected = False

save_restore_labels=[] #is read from ini file

p_d = [0 for n in range(N)]
p_w = [10e-6 for n in range(N)] 
p_e = 0
p_mul = [2**n for n in range(N_out)]
p_la = ['{0:s}'.format(n) for n in list(map(chr, range(65, 65+N)))]

for i in range(N):
    gstring += "P{:d}".format(i)
    if (i<N-1):
        gstring +=','
for i in range(N):
    p="P{:d}={:{width}.{prec}f}".format(i,p_d[i],width=12, prec=9)
    exec(p,globals())

def getcurrent_setting():
    global p_d, w_d, N, N_out, p_w, w_w, p_e, N_out, w_la, p_mul
    for output in range(N_out):
        p_mul[output]=0
        for pulser in range(N):
            if w_mul[output][pulser].value:
                p_mul[output] += 2**pulser
    return {'delay': p_d, 'delay_text':[w_d[i].value for i in range(N)], 'width': p_w,\
            'width_text': [w_w[i].value for i in range(N)],\
            'enable': p_e, 'N': N, 'N_out': N_out, 'label': [w_la[i].value for i in range(N)],\
            'mul': p_mul, 'LoopN': w_pN.value, 'Loopd': w_pd.value}

def setcurrent_setting(d):
    global p_d, p_w, p_la, p_mul, w_d, w_w, p_e, p
    lN=d['N']
    lN_out=d['N_out']
    if (N != lN) or (N_out != lN_out):
        print("Number of Pulser/Outputs incompatible, can't apply settings.") 
    else:
        p_d = d['delay']
        p_w = d['width']
        p_la = d['label']
        p_mul = d['mul']
        p_e = d['enable']
        for i in range(N):
            w_d[i].value = d['delay_text'][i]
            w_w[i].value = d['width_text'][i]
            w_la[i].value = d['label'][i]
        w_pN.value=d['LoopN']
        w_pd.value=d['Loopd']
        update_pulser_ui(textfields=False)
        
   
            
def read_ini():
    global save_restore_labels
    try:
        save_restore_labels=[config['SAVE']['save{:d}'.format(i)] for i in range(N_save)]
    except:
        for i in range(N_save):
            save_restore_labels=['Save Pos {:d}'.format(i) for i in range(N_save)]

def write_ini():
    global save_restore_labels
    save_restore_labels=[w_ls[i].value for i in range(N_save)]
    for i in range(N_save):
        config['SAVE']['save{:d}'.format(i)]=save_restore_labels[i]
    with open(ini_file, 'w') as configfile:
        config.write(configfile)

#save dictionary with all relevant settings        
def save_setting(b):
    global w_ls
    i=w_sb.index(b)
    try:
        with open(w_ls[i].value + '.pkl', 'wb') as f:
            getcurrent_setting()
            pickle.dump(getcurrent_setting(), f)
            write_ini()
    except:
        print("Couldn't write file "+ w_ls[i].value + '.pkl')

#load dictionary with all relevant settings          
def restore_setting(b):
    global w_ls
    i=w_rb.index(b)
    try:
        with open(w_ls[i].value + '.pkl', 'rb') as f:
            new_setting = pickle.load(f)
            setcurrent_setting(new_setting)
    except:
        print("Couldn't read file "+ w_ls[i].value + '.pkl')
    
    
def serial_ports():
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')
    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

def send_pulser(cmd='', show=False):
    global ser
    ser.write(cmd)
    if show:
        print(cmd)
    out = b''
    while ser.inWaiting() > 0:
        out += ser.read(1)
    if out != '':
        if show:
            print (b">>" + out)
    time.sleep(0.001)
    return out

def send_trigger(b):
    send_pulser(b"TRIG\r\n")
    
def send_delay(delay,pulser):
    send_pulser('SET DELAY {:d} {:d}\r\n'.format(int(delay*clk_s),pulser).encode())
    
def send_width(width,pulser):
    send_pulser('SET WIDTH {:d} {:d}\r\n'.format(int(width*clk_s),pulser).encode())     

def send_mux(mux,output):
    send_pulser('SET MUX {:d} {:d}\r\n'.format(int(mux),output).encode())      
    send_pulser('SET MUX {:d} {:d}\r\n'.format(int(mux),output).encode()) #some race condition in FPGA code, need to send twice

def send_enable(b):
    send_pulser('SET ENABLE {:d}\r\n'.format(b).encode())

def nice_time(f):
    if f==0:
        return '0'
    else:
        r=''
        f=int(f*1e9)
        ns=int(f)%1000
        f=int((f-ns)/1000)
        us=int(f)%1000
        f=int((f-us)/1000)
        ms=int(f)%1000
        f=int((f-ms)/1000)

        if f> 0:
            r += '{:d}s'.format(f)
        if ms> 0:
            r += ' {:d}ms'.format(ms)
        if us> 0:
            r += ' {:d}us'.format(us)
        if ns> 0:
            r += ' {:d}ns'.format(ns)
        return r

def update_stats():
    try:
        min_time=(np.array(p_d)[np.array([w_e[i].value for i in range(N)])]).min()
    except ValueError:
        min_time=0
    w_sstart.value='Timing starts at: '+nice_time(min_time)                                                      
    w_send.value='and ends at: '+nice_time(\
            (np.array([w_e[i].value for i in range(N)])*(np.array(p_d)+np.array(p_w))).max())

def update_pulser_ui(textfields=True):
    global w_mul, p_mul, p_d, p_w, p_e
    if connected:
        for i in range(N):
            send_delay(p_d[i],i)
            send_width(p_w[i],i)
            send_enable(p_e)
        for i in range(N_out):
            send_mux(p_mul[i],i)

    for i in range(N):
        if textfields:
            w_d[i].value='{:{width}.{prec}f}'.format(p_d[i], width=12, prec=9)
            w_w[i].value='{:{width}.{prec}f}'.format(p_w[i], width=12, prec=9)
        w_di[i].value='{:{width}.{prec}f}'.format(p_d[i], width=12, prec=9)
        w_wi[i].value='{:{width}.{prec}f}'.format(p_w[i], width=12, prec=9)
        w_e[i].value = ((p_e & 2**i) == 2**i)
    for output in range(N_out):
        for pulser in range(N):
            w_mul[output][pulser].value = False
            if ((p_mul[output] & 2**pulser)== 2**pulser):
                w_mul[output][pulser].value = True
    update_stats()
            
def disable_ui(b):
    a=1 #TODO
    
def my_connect(newvalue):
    global ser, connected
    if (w_connect.value=='connect'):
        try:
            ser = serial.Serial(w_ports.value, baudrate=115200)
            time.sleep(1)
            connected = True
            disable_ui(False)
            update_pulser_ui()
        except:
            w_connect.value='disconnect'
            connected=False
            ser.close()
            disable_ui(True)
            print("ERROR: couldn't open serial port.")
            time.sleep(1)
            #reread serial ports
            myserialports=serial_ports()
            w_ports.options=myserialports
            w_ports.value=myserialports[-1]
    else:
        ser.close()
        connected = False
        disable_ui(True)

def dw_change(change):
    exec(gstring,globals())
    delay=True
    try:
        i=w_d.index(change['owner'])
        inp_i=w_d[i]
        out_i=w_di[i]
        check = lambda a : (a<100) and (a>=0)      
    except ValueError:
        i=w_w.index(change['owner'])
        inp_i=w_w[i]
        out_i=w_wi[i]
        check = lambda a : (a<100) and (a>=10e-9)
        delay=False
    try:
        myf=float(eval(change['new']))
        if check(myf):
            mys='{:{width}.{prec}f}'.format(myf, width=12, prec=9)
            if delay:
                p="P{:d}={:{width}.{prec}f}".format(i,myf,width=12, prec=9)
                exec(p,globals())
                if connected:
                    send_delay(myf,i)
                p_d[i]=myf
            else:
                if connected:
                    send_width(myf,i)
                p_w[i]=myf
        else:
            raise
    except:
        inp_i.remove_class('data_input')
        inp_i.add_class('data_wrong')
        pass
    else:
        out_i.value=mys
        inp_i.remove_class('data_wrong')
        inp_i.add_class('data_input')
    update_stats()

def mul_change(change):
    global p_mul, w_mul
    for output in range(N_out):
        #w_mul[Output][Pulser]
        mux = 0
        for pulser in range(N):
            mux += (2**pulser)*(w_mul[output][pulser].value==True) 
        #print(output,mux)
        if connected:
            send_mux(mux,output)

        
def en_change(change):
    global p_e, w_e
    #i=w_e.index(change['owner'])
    b=0;
    for i in range(N):
        b+=(w_e[i].value==True)*2**i
    if connected:
        send_enable(b)
    p_e=b
    update_stats()
    

def pl_change(b):
    try:
        myd=int(w_pd.value)
        myN=int(w_pN.value)
    except:
        myd=0
        myN=2
    finally:
        if (myd>100000) or (myd<0):
            myd=0
        w_pd.value="{:d}".format(myd)
        if (myN>1000) or (myN<2):
             myN=2
        w_pN.value="{:d}".format(myN)
        w_pl2.value="Run 0/{:d}".format(myN)

        
def process_loop(b):
    max_run=int(w_pN.value)
    run=1
    delay=float(w_pd.value)
    while run<=max_run:
        w_pl2.value="Run {:d}/{:d}".format(run,max_run)
        send_trigger('x')
        run+=1
        time.sleep(delay/1000.)
    w_pl2.value="Run 0/{:d}".format(max_run)

##THIS IS TO SPEED UP DEVELOPMENT BUT MUST BE TAKEN OUT IN FINAL PROGRAM    
try:
    myserialports
except NameError:
    myserialports=serial_ports()


    
    
    
#Read INI file with basic configurations
read_ini()

w_connect=widgets.ToggleButtons(description='Pulser:',\
                               options=['connect', 'disconnect'], value="disconnect")
w_ports=widgets.Dropdown(options=myserialports,\
                         value=myserialports[-1], description='Port:', disabled=False)
w_connect.observe(my_connect, names='value')

#Statistics
w_sstart=widgets.Label('Timing starts at: ')
w_sstart.layout.max_width="35ex"
w_send=widgets.Label(' and ends at: ')
w_send.layout.max_width="35ex"

#process loop
w_pd=widgets.Text(description="Wait:",value="0",placeholder='Delay (ms)',disabled=False)
w_pd.layout.max_width="20ex"
w_pd.observe(pl_change, names='value')
w_pl1=widgets.Label(value="ms")
w_pN=widgets.Text(description="N:",value="2",placeholder='Runs',disabled=False)
w_pN.layout.max_width="20ex"
w_pN.observe(pl_change, names='value')
w_pl2=widgets.Label(value="Run 0/2")
w_pB=widgets.Button(description='Run Loop', button_style='danger')
w_pB.on_click(process_loop)
w_pl_box=widgets.HBox([w_pd,w_pl1,w_pN,w_pl2,w_pB])




w_l=[]; w_e=[]; w_d=[]; w_di=[]; w_w=[]; w_wi=[]; w_la=[];
for n in range(N):
    #Label
    w_l.append(widgets.Label('P{:d}'.format(n)))
    w_l[n].layout.width="4ex"
    #Textlabel
    w_la.append(widgets.Text(value=p_la[n],layout=Layout(width='8ex')))
    #Enable
    if (p_e & 2**n):
        val=True
    else:
        val=False
    w_e.append(widgets.Checkbox(value=val, disabled=False, indent=False))
    w_e[n].layout.max_width="4ex"
    w_e[n].observe(en_change, names='value')
    #Input Delay
    w_d.append(widgets.Text(value="0",placeholder='Pulser 1',disabled=False))
    w_d[n].layout.max_width="20ex"
    w_d[n].observe(dw_change, names='value')
    w_d[n].add_class('data_input')
    #Evaled Delay
    w_di.append(widgets.Text(value="0",disabled=True))
    w_di[n].layout.width="20ex"
    w_di[n].add_class('data_display')
    #Input Width
    w_w.append(widgets.Text(value="0",placeholder='Pulser 1',disabled=False))
    w_w[n].layout.max_width="20ex"
    w_w[n].observe(dw_change, names='value')
    w_w[n].add_class('data_input')
    #Evaled Width
    w_wi.append(widgets.Text(value="0",disabled=True))
    w_wi[n].layout.max_width="20ex"
    w_wi[n].add_class('data_display')

disable_ui(True)
w_pps=[]
for n in range(N):
    w_pps.append(widgets.HBox([w_la[n],\
                               widgets.VBox([widgets.HBox([w_l[n],w_d[n],w_w[n]]),\
                               widgets.HBox([w_e[n],w_di[n],w_wi[n]])])],\
                               layout=Layout(border='solid 1px')));
    
w_trigger=widgets.Button(description='Trigger', button_style='danger')
w_trigger.on_click(send_trigger)

ht=widgets.HTML(data_input_style)

#Multiplex content
w_mul=[]
m=[]
l=[widgets.Label(value=str(i)) for i in range(N)]
l=[widgets.Label(value=' ')]+l
ll=widgets.GridBox(l, layout=widgets.Layout(grid_template_columns="repeat({:d}, 4ex)".format(N+1)))

#w_mul[Output][Pulser]
for i in range(N_out):
    w_mul.append([widgets.Checkbox(value=(i == j), description='',\
                  indent=False, layout=Layout(height='2.5ex')) for j in range(N)])
    ka=[widgets.Label(value=str(i))]+w_mul[i]
    m.append(widgets.GridBox(ka, layout=Layout(gap_row_gap='0ex', grid_template_columns="repeat({:d}, 4ex)".format(N+1))))
for output in range(N_out):
    for pulser in range(N):
        w_mul[output][pulser].observe(mul_change, names='value')
        
#w_saves
w_ls=[]
w_sb=[]
w_rb=[]
for i in range(N_save):
    w_ls.append(widgets.Text(value=save_restore_labels[i],\
                placeholder='Name Position {:d}'.format(i),disabled=False))
    w_ls[i].layout.max_width="20ex"
    w_sb.append(widgets.Button(description='Save'.format(i)))
    w_sb[i].on_click(save_setting)
    w_sb[i].layout.max_width="10ex"
    w_rb.append(widgets.Button(description='Restore'.format(i)))
    w_rb[i].on_click(restore_setting)
    w_rb[i].layout.max_width="10ex"
    
        
s=widgets.VBox([ll]+[m[i] for i in range(N_out)])    
tab_contents = ['Delay & Width', 'Multiplex', 'Save & Restore']

VB1=widgets.VBox([w_pps[i] for i in range(int(N/2))])
VB2=widgets.VBox([w_pps[i+int(N/2)] for i in range(int(N/2))])
VB3=widgets.HBox([widgets.VBox([w_ls[i] for i in range(N_save)]),\
                  widgets.VBox([w_sb[i] for i in range(N_save)]),\
                  widgets.VBox([w_rb[i] for i in range(N_save)])])
                   
children=[widgets.HBox([VB1,VB2]),widgets.VBox([widgets.Label(value='Pulser'),s]),VB3]
tab = widgets.Tab()
tab.children = children

for i in range(len(children)):
    tab.set_title(i, tab_contents[i])

#Starting User Interface
display.display(widgets.VBox([widgets.HBox([ht,w_ports,w_connect,w_trigger]),widgets.HBox([w_sstart,w_send])]))
display.display(w_pl_box)
display.display(tab)


update_pulser_ui()